# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas-docs.github.io/pandas-docs-travis/io.html#json
+ data source: http://jsonstudio.com/resources/
****

In [1]:
import pandas as pd

## imports for Python, Pandas

In [2]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas-docs.github.io/pandas-docs-travis/io.html#normalization

In [3]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [4]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

,name,population
0,Dade,12345
1,Broward,40000
2,Palm Beach,60000
3,Summit,1234
4,Cuyahoga,1337


In [5]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

,name,population,info.governor,state,shortname
0,Dade,12345,Rick Scott,Florida,FL
1,Broward,40000,Rick Scott,Florida,FL
2,Palm Beach,60000,Rick Scott,Florida,FL
3,Summit,1234,John Kasich,Ohio,OH
4,Cuyahoga,1337,John Kasich,Ohio,OH


****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [6]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

[{u'_id': {u'$oid': u'52b213b38594d8a2be17c780'},
  u'approvalfy': 1999,
  u'board_approval_month': u'November',
  u'boardapprovaldate': u'2013-11-12T00:00:00Z',
  u'borrower': u'FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA',
  u'closingdate': u'2018-07-07T00:00:00Z',
  u'country_namecode': u'Federal Democratic Republic of Ethiopia!$!ET',
  u'countrycode': u'ET',
  u'countryname': u'Federal Democratic Republic of Ethiopia',
  u'countryshortname': u'Ethiopia',
  u'docty': u'Project Information Document,Indigenous Peoples Plan,Project Information Document',
  u'envassesmentcategorycode': u'C',
  u'grantamt': 0,
  u'ibrdcommamt': 0,
  u'id': u'P129828',
  u'idacommamt': 130000000,
  u'impagency': u'MINISTRY OF EDUCATION',
  u'lendinginstr': u'Investment Project Financing',
  u'lendinginstrtype': u'IN',
  u'lendprojectcost': 550000000,
  u'majorsector_percent': [{u'Name': u'Education', u'Percent': 46},
   {u'Name': u'Education', u'Percent': 26},
   {u'Name': u'Public Administration, Law, and Ju

In [7]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{u'$oid': u'52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{u'Percent': 100, u'Name': u'Education for all'}","[{u'code': u'65', u'name': u'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{u'$oid': u'52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{u'Percent': 30, u'Name': u'Other economic man...","[{u'code': u'24', u'name': u'Other economic ma...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [8]:
#### Question 1 ####

# Building a pandas data frame from json file
json_df = pd.read_json('data/world_bank_projects.json')
# json_df.head()

# Group by countryshortname
j = json_df
j = j.groupby(['countryshortname']).size()

# Select the first 10 countries
j = j.sort_values(ascending=False).head(10)
j

countryshortname
Indonesia             19
China                 19
Vietnam               17
India                 16
Yemen, Republic of    13
Nepal                 12
Bangladesh            12
Morocco               12
Mozambique            11
Africa                11
dtype: int64

In [9]:
#### Question 2 ####

# Load json as string
data = json.load((open('data/world_bank_projects.json')))

## Use normalization to create tables from nested element
d_norm = json_normalize(data, 'mjtheme_namecode')
d_norm

# Group by code
d2 = d_norm.groupby(['code']).size()

# Select the first 10 major projects
d2 = d2.sort_values(ascending=False).head(10)
d2 = d2.to_frame().reset_index()
d2

# Grouping only by 'code' enables use to take in consideration the entries
# where the name is missing.


,code,0
0,11,250
1,10,216
2,8,210
3,2,199
4,6,168
5,4,146
6,7,130
7,5,77
8,9,50
9,1,38


In [10]:
# If we want to see what it looks like with this missing data we can do the following

# Group by 'code' and 'name'
d2_2 = d_norm.groupby(['code', 'name']).size()

# Select the first 10 major projects
d2_2 = d2_2.sort_values(ascending=False).head(10)
d2_2 = d2_2.to_frame().reset_index()
d2_2

,code,name,0
0,11,Environment and natural resources management,223
1,10,Rural development,202
2,8,Human development,197
3,2,Public sector governance,184
4,6,Social protection and risk management,158
5,4,Financial and private sector development,130
6,7,Social dev/gender/inclusion,119
7,5,Trade and integration,72
8,9,Urban development,47
9,1,Economic management,33


In [11]:
# Using the merge command we can combine these two tables and have the name 
# of the themes next to the code and count values

final_table = pd.merge(d2, d2_2, how='inner', left_on='code', right_on='code', left_index=False, right_index=False)

new_columns = final_table.columns.values
new_columns[1] = 'count'
final_table.columns = new_columns

del final_table['0_y']

final_table

,code,count,name
0,11,250,Environment and natural resources management
1,10,216,Rural development
2,8,210,Human development
3,2,199,Public sector governance
4,6,168,Social protection and risk management
5,4,146,Financial and private sector development
6,7,130,Social dev/gender/inclusion
7,5,77,Trade and integration
8,9,50,Urban development
9,1,38,Economic management


In [11]:
#### Question 3 ####

# STEP 1

# Build a data frame which maps the project code and the project name
# ignoring the entries where the name is missing.
d3 = d_norm[d_norm['name'] != ""].groupby(['code', 'name']).size()

d3 = d3.reset_index()

# Rename the third column
new_columns = d3.columns.values
new_columns[2] = 'count'
d3.columns = new_columns

d3


,code,name,count
0,1,Economic management,33
1,10,Rural development,202
2,11,Environment and natural resources management,223
3,2,Public sector governance,184
4,3,Rule of law,12
5,4,Financial and private sector development,130
6,5,Trade and integration,72
7,6,Social protection and risk management,158
8,7,Social dev/gender/inclusion,119
9,8,Human development,197


In [193]:
# STEP 2

# Loop through all the different 'mjtheme_namecode' of the original data frame
for items in json_df['mjtheme_namecode']:
    for item in items:
        # Check if the current name is a string (not necessary)
        if isinstance(item['name'], basestring):
            # Check if the 'name' property is missing for this entries
            if item['name'] == "":
                # If it is, we fill it using the table built in STEP 1
                mask = d3['code'] == item['code']
                item['name'] = d3[mask]['name'].iloc[0]
            
json_df['mjtheme_namecode'].head()

0    [{u'code': u'8', u'name': u'Human development'...
1    [{u'code': u'1', u'name': u'Economic managemen...
2    [{u'code': u'5', u'name': u'Trade and integrat...
3    [{u'code': u'7', u'name': u'Social dev/gender/...
4    [{u'code': u'5', u'name': u'Trade and integrat...
Name: mjtheme_namecode, dtype: object

In [174]:
new_json = json_df.to_json()
new_json

'{"_id":{"0":{"$oid":"52b213b38594d8a2be17c780"},"1":{"$oid":"52b213b38594d8a2be17c781"},"2":{"$oid":"52b213b38594d8a2be17c782"},"3":{"$oid":"52b213b38594d8a2be17c783"},"4":{"$oid":"52b213b38594d8a2be17c784"},"5":{"$oid":"52b213b38594d8a2be17c785"},"6":{"$oid":"52b213b38594d8a2be17c786"},"7":{"$oid":"52b213b38594d8a2be17c787"},"8":{"$oid":"52b213b38594d8a2be17c788"},"9":{"$oid":"52b213b38594d8a2be17c789"},"10":{"$oid":"52b213b38594d8a2be17c78a"},"11":{"$oid":"52b213b38594d8a2be17c78b"},"12":{"$oid":"52b213b38594d8a2be17c78c"},"13":{"$oid":"52b213b38594d8a2be17c78d"},"14":{"$oid":"52b213b38594d8a2be17c78e"},"15":{"$oid":"52b213b38594d8a2be17c78f"},"16":{"$oid":"52b213b38594d8a2be17c790"},"17":{"$oid":"52b213b38594d8a2be17c791"},"18":{"$oid":"52b213b38594d8a2be17c792"},"19":{"$oid":"52b213b38594d8a2be17c793"},"20":{"$oid":"52b213b38594d8a2be17c794"},"21":{"$oid":"52b213b38594d8a2be17c795"},"22":{"$oid":"52b213b38594d8a2be17c796"},"23":{"$oid":"52b213b38594d8a2be17c797"},"24":{"$oid":"52b